In [1]:
import codecs
import json
import numpy as np
import sklearn

In [2]:
from Scripts.ProcessingEmbeddings import *
import Scripts.utils as utils

In [4]:
#Creating an embeddings object: 400k words, 50 dimensions
glove=Embeddings('Data/glove-wiki-gigaword-300.txt', gensim=False)
#word2vec=Embeddings('Data/word2vec-google-news-300.txt', gensim=False)

Loading Data/glove-wiki-gigaword-300.txt embeddings
vectors shape: (400000, 300), word2idx length: 400000, vocab length: 400000


In [5]:
#Getting the attributes of the embeddings 
vectors=glove.vectors
word2idx=glove.word2idx
vocab=glove.words
dict_vectors = glove.get_word_vector_dict()


In [6]:
#Verifying the attributes: there shouldn't be any null values
np.isnan(vectors).any()

False

In [8]:
#Removing puntuation and numbers from the embeddings
vocab_cleaned, vectors_cleaned, word2idx_cleaned, dict_vec_cleaned = glove.limit_vocab(
    vectors, word2idx, vocab)


100%|██████████| 400000/400000 [00:00<00:00, 605578.16it/s]


Size of limited vocabulary: 327185


In [9]:
#Checking if vector are normalized
np.apply_along_axis(np.linalg.norm, 1, vectors_cleaned)
#They are not!

array([5.18139682, 5.45449219, 5.91459155, ..., 5.73745479, 6.25541744,
       3.3774816 ])

## Hard-Debias Algorithm

#### Preliminaries

In [10]:
from Scripts.HardDebias import *

In [11]:
#Gender specific vocabulary from Bolukbasi's paper.
gender_specific = []
female_vocab = []
male_vocab = []
with open('./Data/male_word_file.txt') as f:
    male_vocab = [line.strip() for line in f]

with open('./Data/female_word_file.txt') as f:
    for l in f:
        female_vocab.append(l.strip())

gender_specific = female_vocab+female_vocab

with codecs.open('./Data/gender_specific_full.json') as f:
    gender_specific.extend(json.load(f))


In [79]:
#Getting the definitional sets to calculate afterwards the gender direction. The first 10 gender sets were proposed by Bolukbasi et al. (2016)
#Definitional sets for race where proposed by Manzini et al. in Multiclass debiasing of embeddings: https://github.com/TManzini/DebiasMulticlassWordEmbedding/blob/master/Debiasing/data/vocab/race_attributes_optm.json

def_sets={
    "gender" : [
    ['she', 'he'], ['herself', 'himself'], ['her', 'his'], ['daughter', 'son'], ['girl', 'boy'],
    ['mother', 'father'], ['woman', 'man'], ['mary', 'john'], ['gal', 'guy'], ['female', 'male'],['aunt', 'uncle']],
    
    "race":[
		["black", "caucasian", "asian", "hispanic"],
		["african", "caucasian", "asian", "hispanic"],
		["black", "white", "asian", "latino"],
		["africa", "europe", "asia", "mexico"],
		["africa", "america", "china", "latin-america"],
    ]
	}

#Equalizing pairs for gender debiasing were first published by Bolukbasi et al. in https://github.com/tolga-b/debiaswe/blob/master/data/equalize_pairs.json
# Equalizing sets for race where defined by Manzini as equal to the defining set (Manzini et al., 2019.p.3)
equalizing_lists = {
    "gender": [
        ["monastery", "convent"], ["spokesman", "spokeswoman"], [
            "Catholic_priest", "nun"], ["Dad", "Mom"], ["Men", "Women"],
        ["councilman", "councilwoman"], ["grandpa", "grandma"], [
            "grandsons", "granddaughters"], ["prostate_cancer", "ovarian_cancer"],
        ["testosterone", "estrogen"], ["uncle", "aunt"], [
            "wives", "husbands"], ["Father", "Mother"], ["Grandpa", "Grandma"],
        ["He", "She"], ["boy", "girl"], ["boys", "girls"], ["brother", "sister"], [
            "brothers", "sisters"], ["businessman", "businesswoman"],
        ["chairman", "chairwoman"], ["colt", "filly"], ["congressman",
                                                        "congresswoman"], ["dad", "mom"], ["dads", "moms"], ["dudes", "gals"],
        ["ex_girlfriend", "ex_boyfriend"], ["father", "mother"], [
            "fatherhood", "motherhood"], ["fathers", "mothers"], ["fella", "granny"],
        ["fraternity", "sorority"], ["gelding", "mare"], ["gentleman", "lady"], [
            "gentlemen", "ladies"], ["grandfather", "grandmother"],
        ["grandson", "granddaughter"], ["he", "she"], ["himself", "herself"], [
            "his", "her"], ["king", "queen"], ["kings", "queens"],
        ["male", "female"], ["males", "females"], ["man", "woman"], [
            "men", "women"], ["nephew", "niece"], ["prince", "princess"],
        ["schoolboy", "schoolgirl"], ["son", "daughter"], ["sons", "daughters"], ["twin_brother", "twin_sister"]],

    "race": [
        ["black", "caucasian", "asian"],
      	["african", "caucasian", "asian"],
      	["black", "white", "asian"],
      	["africa", "america", "asia"],
      	["africa", "america", "china"],
      	["africa", "europe", "asia"]
    ]}

#Some of the words were taken from the analogies' templates from Cheng and Manzini.
#The list is not the same, however, because some of the words were not neutral, but carried some
#relation to the social categories.
neutral_words = ["manager", "executive", "doctor", "lawyer", "programmer",
                 "scientist", "soldier", "supervisor", "rancher", "janitor",
                 "firefighter", "officer", "secretary", "nurse", "clerk", "artist",
                 "homemaker", "dancer", "singer", "librarian", "maid", "hairdresser", "stylist",
                 "receptionist", "counselor", "leader", "farmer",
                 "engineer", "laborer", "teacher",
                 "slave", "musician", "runner", "criminal", "homeless",
                 "greedy", "cheap", "hairy", "liberal",
                 "judgemental", "conservative", "familial",
                 "violent", "terrorist", "dirty", "uneducated", "educated"]


#However, also the vocabulary without the gendered words from the list can be conceived as neutral, according to Bolukbasi et al.


In [98]:
#Lists of names for validation
#Adapted from Speer's tutorial on racism in sentiment analysis. http://blog.conceptnet.io/posts/2017/how-to-make-a-racist-ai-without-really-trying/
names_ethnicity = {
    # The first two lists are from the Caliskan et al. appendix describing the
    # Word Embedding Association Test.
    'White': [
        'Adam', 'Chip', 'Harry', 'Josh', 'Roger', 'Alan', 'Frank', 'Ian', 'Justin',
        'Ryan', 'Andrew', 'Fred', 'Jack', 'Matthew', 'Stephen', 'Brad', 'Greg', 'Jed',
        'Paul', 'Todd', 'Brandon', 'Hank', 'Jonathan', 'Peter', 'Wilbur', 'Amanda',
        'Courtney', 'Heather', 'Melanie', 'Sara', 'Amber', 'Crystal', 'Katie',
        'Meredith', 'Shannon', 'Betsy', 'Donna', 'Kristin', 'Nancy', 'Stephanie',
        'Bobbie-Sue', 'Ellen', 'Lauren', 'Peggy', 'Sue-Ellen', 'Colleen', 'Emily',
        'Megan', 'Rachel', 'Wendy'
    ],

    'Black': [
        'Alonzo', 'Jamel', 'Jamal', 'Lerone', 'Percell', 'Theo', 'Alphonse', 'Jerome',
        'Leroy', 'Rasaan', 'Torrance', 'Darnell', 'Lamar', 'Lionel', 'Rashaun',
        'Tyree', 'Deion', 'Lamont', 'Malik', 'Terrence', 'Tyrone', 'Everol',
        'Lavon', 'Marcellus', 'Terryl', 'Wardell', 'Aiesha', 'Lashelle', 'Nichelle',
        'Shereen', 'Temeka', 'Ebony', 'Latisha', 'Shaniqua', 'Tameisha', 'Teretha',
        'Jasmine', 'Latonya', 'Shanise', 'Tanisha', 'Tia', 'Lakisha', 'Latoya',
        'Sharise', 'Tashika', 'Yolanda', 'Lashandra', 'Malika', 'Shavonn',
        'Tawanda', 'Yvette'
    ],
    
    # This list comes from statistics about common Hispanic-origin names in the US.
    'Hispanic': [
        'Juan', 'José', 'Miguel', 'Luís', 'Jorge', 'Santiago', 'Matías', 'Sebastián',
        'Mateo', 'Nicolás', 'Alejandro', 'Samuel', 'Diego', 'Daniel', 'Tomás',
        'Juana', 'Ana', 'Luisa', 'María', 'Elena', 'Sofía', 'Isabella', 'Valentina',
        'Camila', 'Valeria', 'Ximena', 'Luciana', 'Mariana', 'Victoria', 'Martina'
    ],
    
   
}
#Following Bolukbasi et al. Implementing notebook: https://github.com/tolga-b/debiaswe/blob/master/tutorial_example1.ipynb
names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
         "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
         "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
         "Leroy", "Brad", "Jermaine"]
#names_group1 = [names[2 * i] for i in range(len(names) // 2)]
#names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]



In [14]:
#Preparing the definite sets for debiasing
def_set_gender=utils.prepare_def_sets_subspace(def_sets["gender"])
def_set_race=utils.prepare_def_sets_subspace(def_sets["race"])

In [43]:
deb_vect_gender, deb_vocab_gender, deb_word2idx_gender, deb_dict_gender = hard_debias(vectors,
                                                                                      dict_vectors,
                                                                                      word2idx_cleaned,
                                                                                      vocab_cleaned,
                                                                                      equalizing_lists['gender'],
                                                                                      def_set_gender,
                                                                                      1,
                                                                                      normalize_dir=False,
                                                                                      normalize=None,
                                                                                      centralizing=True)


Length of vectors set: 22
Running PCA with 1 components


## Intersectionality study
Bias is not binary nor one dimensional, if clusters are shuffled, what is the effect of the shuffling on other social classes? Is there a way to measure it?

In [15]:
#Find the words in the female_vocab that are also in the embeddings
female_words_emb=[word for word in female_vocab if word in dict_vec_cleaned.keys()]
male_words_emb = [word for word in male_vocab if word in dict_vec_cleaned.keys()]

print('Number of female words in embeddings:', len(female_words_emb))
print('Number of male words in embeddings:', len(male_words_emb))


Number of female words in embeddings: 195
Number of male words in embeddings: 204


In [16]:
#getting the gender directions
gen_dir_centralized=identify_bias_subspace(dict_vec_cleaned, def_set_gender, 1, centralizing=True)
#flattening them
gen_dir_centralized_flat=np.squeeze(gen_dir_centralized)


Length of vectors set: 22
Running PCA with 1 components


In [44]:
from Scripts.Evaluation import compute_gender_simple_bias, compute_similarity_to_bias_direction
similarity_centralized=compute_similarity_to_bias_direction(dict_vec_cleaned, gen_dir_centralized_flat)
simple_gender_bias=compute_gender_simple_bias(dict_vec_cleaned, dict_vec_cleaned['he'], dict_vec_cleaned['she'])
deb_similarity_centralized=compute_similarity_to_bias_direction(deb_dict_gender, gen_dir_centralized_flat)
deb_simple_gender_bias=compute_gender_simple_bias(deb_dict_gender, deb_dict_gender['he'], deb_dict_gender['she'])

In [26]:
#Find the words in the female_vocab that are also in the embeddings
names_white_emb = [
    word.lower() for word in names_ethnicity['White'] if word.lower() in dict_vec_cleaned.keys()]
names_black_emb = [
    word.lower() for word in names_ethnicity['Black'] if word.lower() in dict_vec_cleaned.keys()]

names_hispanic_emb = [
    word.lower() for word in names_ethnicity['Hispanic'] if word.lower() in dict_vec_cleaned.keys()]


print('Number of white names in embeddings:', len(names_white_emb))
print('Number of black names in embeddings:', len(names_black_emb))
print('Number of hispanic names in embeddings:', len(names_hispanic_emb))



Number of white names in embeddings: 48
Number of black names in embeddings: 38
Number of hispanic names in embeddings: 30


In [45]:
import pandas as pd
#get a dataframe with the bias scores of the female_words_emb and male_words_emb in the similarity, similarity_centralized and simple_gender_bias


def get_df_bias_scores(word_list, similarity_centralized, simple_bias_score):
    scores = {}
    for word in word_list:
        scores[word] = {"centralized_similarity_score": (similarity_centralized[word]),
                        "simple_bias_score": simple_bias_score[word]}
    df = pd.DataFrame.from_dict(scores, orient='index')
    return df


In [58]:
df_names_white=get_df_bias_scores(names_white_emb, similarity_centralized,simple_gender_bias)
df_names_black=get_df_bias_scores(names_black_emb, similarity_centralized,simple_gender_bias)
df_names_hispanic=get_df_bias_scores(names_hispanic_emb, similarity_centralized,simple_gender_bias)

#now the dataframe for the debiased embeddings
df_names_white_deb=get_df_bias_scores(names_white_emb, deb_similarity_centralized,deb_simple_gender_bias)
df_names_black_deb = get_df_bias_scores(names_black_emb, deb_similarity_centralized,deb_simple_gender_bias)
df_names_hispanic_deb = get_df_bias_scores(
    names_hispanic_emb, deb_similarity_centralized, deb_simple_gender_bias)

#merging the dataframes
df_names_white_merged=pd.merge(df_names_white, df_names_white_deb, left_index=True, right_index=True, suffixes=('_orig', '_deb'))
df_names_black_merged=pd.merge(df_names_black, df_names_black_deb, left_index=True, right_index=True, suffixes=('_orig', '_deb'))
df_names_hispanic_merged=pd.merge(df_names_hispanic, df_names_hispanic_deb, left_index=True, right_index=True, suffixes=('_orig', '_deb'))


df_names_white_merged


,centralized_similarity_score_orig,simple_bias_score_orig,centralized_similarity_score_deb,simple_bias_score_deb
adam,-0.185898,-0.094373,-1.402362e-07,1.493309e-09
chip,-0.044571,-0.020428,-7.884275e-08,5.179480e-09
harry,-0.071752,0.006493,-1.990548e-07,3.913110e-09
josh,-0.096062,-0.014186,-1.646718e-07,6.410092e-09
roger,-0.231225,-0.110803,-1.328456e-07,7.128524e-09
alan,-0.129366,-0.048589,-9.871449e-08,5.221612e-09
frank,-0.176201,-0.068455,-8.353230e-09,1.151710e-09
ian,-0.137793,-0.049633,-2.072702e-08,1.013679e-08
justin,-0.122327,-0.019525,-1.249698e-07,-5.482692e-09
ryan,-0.077191,-0.017689,-1.789196e-07,1.025711e-08


In [69]:
# To do: plot the barplots of similarity and simple bias scores faceted for original and debiased embeddings
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()
#fig= make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(
    x=df_names_white_merged.index,
    y=df_names_white_merged['centralized_similarity_score_orig'],
    name='Original',
    marker_color='indianred'),
    #secondary_y=False
)
fig.add_trace(go.Bar(x=df_names_white_merged.index,
                     y=df_names_white_merged['simple_bias_score_orig'],
                     name='Original_ simple',
                     marker_color='green'),
                     #secondary_y=False
                     )


fig1 = go.Figure()
fig1.add_trace(go.Bar(x=df_names_white_merged.index,
                        y=df_names_white_merged['simple_bias_score_deb'],
                        name='Debiased_ simple',
                        marker_color='lightgreen'), 
                        #secondary_y=True
                        )


fig1.add_trace(go.Bar(
    x=df_names_white_merged.index,
    y=df_names_white_merged['centralized_similarity_score_deb'],
    name='Debiased',
    marker_color='lightsalmon'),
    #secondary_y=True
)

fig.show()
fig1.show()

In [70]:
#plot a bar plot of the top 20 most biased words with all the scores of the three methods
import plotly_express as px


def plot_top_biased_words(df, n_words=20):
    df_top = df.head(n_words)
    #remove the simple_bias_score column
    #df_top = df_top.drop(columns=['simple_bias_score'])
    df_top = df_top.reset_index()
    df_top = df_top.rename(columns={'index': 'word'})
    df_top = df_top.melt(
        id_vars=['word'], var_name='score_type', value_name='score')
    fig = px.bar(df_top, x="score", y="word",
                 color="score_type", barmode="group", orientation='h',
                 height=1000, width=800, title="Top biased words")
   
    
    fig.show()


plot_top_biased_words(df_names_white, n_words=50)
plot_top_biased_words(df_names_black, n_words=50)
plot_top_biased_words(df_names_hispanic, n_words=50)


1.2 Neutral words associated with ethnic names

In [72]:
from Scripts.Evaluation import *

In [87]:
def get_frequency_neutral_words(names, neutral_words, dict_vect_debiased, vocab_debiased, vectors_debiased, w2i_debiased, neighbours_num=50):
    """"
    Function to get the frequency of the original neighbors among the 50 nearest neighbors of selected words
    :param list_words: list of words to compute the bias for
    :param list_neigh: list of the neighbors for each word of the dictionary k_neigh
    :param dict_vect_debiased: dictionary of words and their embeddings
    :param vocab_debiased: list of words in the vocabulary
    :param vectors_debiased: list of embeddings
    :param w2i_debiased: dictionary of words and their indices
    :param neighbours_num: number of neighbors to find
    :return: list of the frequency of the original neighbors among the 50 nearest neighbors of selected words
    """
    scores = []
    for word in tqdm(set(names)):
        which_words=[]
        #get the top 50 neighbors of the word
        _, top = get_topK_neighbors(word, dict_vect_debiased, vocab_debiased, vectors_debiased, w2i_debiased,
                                    k=neighbours_num)

        count = 0
        #check if the original neighbors are in the top 50
        for t in top:
            if t in set(neutral_words):
                print(t)
                count += 1
                which_words.append(t)

        scores.append([word, which_words, count, count/neighbours_num])
        #print(top)
    return scores


In [88]:
neutral_words

['manager',
 'executive',
 'doctor',
 'lawyer',
 'programmer',
 'scientist',
 'soldier',
 'supervisor',
 'rancher',
 'janitor',
 'firefighter',
 'officer',
 'secretary',
 'nurse',
 'clerk',
 'artist',
 'homemaker',
 'dancer',
 'singer',
 'librarian',
 'maid',
 'hairdresser',
 'stylist',
 'receptionist',
 'counselor',
 'leader',
 'farmer',
 'engineer',
 'laborer',
 'teacher',
 'slave',
 'musician',
 'runner',
 'criminal',
 'homeless',
 'greedy',
 'cheap',
 'hairy',
 'liberal',
 'judgemental',
 'conservative',
 'familial',
 'violent',
 'terrorist',
 'dirty',
 'uneducated',
 'educated']

In [91]:
#frequencies of neighbors
neutral_on_white=get_frequency_neutral_words(
    names_white_emb, neutral_words, dict_vectors, vocab_cleaned, vectors_cleaned, word2idx_cleaned, neighbours_num=100)

neutral_on_white_deb = get_frequency_neutral_words(
    names_white_emb, neutral_words, deb_dict_gender, deb_vocab_gender, deb_vect_gender, deb_word2idx_gender, neighbours_num=100)


 21%|██        | 10/48 [00:18<01:09,  1.83s/it]

singer


 52%|█████▏    | 25/48 [00:45<00:41,  1.81s/it]

executive
manager


 71%|███████   | 34/48 [01:01<00:25,  1.80s/it]

...

 62%|██████▎   | 30/48 [00:48<00:29,  1.62s/it]

counselor


 71%|███████   | 34/48 [00:55<00:22,  1.62s/it]

dirty


100%|██████████| 48/48 [01:18<00:00,  1.63s/it]


In [99]:
neutral_on_black = get_frequency_neutral_words(
    names_black_emb, neutral_words, dict_vectors, vocab_cleaned, vectors_cleaned, word2idx_cleaned, neighbours_num=100)


neutral_on_black_deb = get_frequency_neutral_words(
    names_black_emb, neutral_words, deb_dict_gender, deb_vocab_gender, deb_vect_gender, deb_word2idx_gender, neighbours_num=100)


 89%|████████▉ | 34/38 [00:54<00:06,  1.60s/it]

counselor


100%|██████████| 38/38 [01:00<00:00,  1.60s/it]


In [94]:
neutral_on_black_deb


[['rashaun', [], 0, 0.0],
 ['theo', [], 0, 0.0],
 ['ebony', [], 0, 0.0],
 ['malik', [], 0, 0.0],
 ['marcellus', [], 0, 0.0],
 ['latisha', [], 0, 0.0],
 ['lakisha', [], 0, 0.0],
 ['temeka', [], 0, 0.0],
 ['shereen', [], 0, 0.0],
 ['tyree', [], 0, 0.0],
 ['malika', [], 0, 0.0],
 ['yolanda', [], 0, 0.0],
 ['tyrone', [], 0, 0.0],
 ['jasmine', [], 0, 0.0],
 ['deion', [], 0, 0.0],
 ['leroy', [], 0, 0.0],
 ['jamel', [], 0, 0.0],
 ['lerone', [], 0, 0.0],
 ['terrence', [], 0, 0.0],
 ['tawanda', [], 0, 0.0],
 ['yvette', [], 0, 0.0],
 ['wardell', [], 0, 0.0],
 ['lamont', [], 0, 0.0],
 ['terryl', [], 0, 0.0],
 ['latoya', [], 0, 0.0],
 ['torrance', [], 0, 0.0],
 ['nichelle', [], 0, 0.0],
 ['tia', [], 0, 0.0],
 ['tanisha', [], 0, 0.0],
 ['darnell', [], 0, 0.0],
 ['latonya', [], 0, 0.0],
 ['alonzo', [], 0, 0.0],
 ['lamar', [], 0, 0.0],
 ['jerome', ['counselor'], 1, 0.01],
 ['shaniqua', [], 0, 0.0],
 ['alphonse', [], 0, 0.0],
 ['lavon', [], 0, 0.0],
 ['lionel', [], 0, 0.0]]

In [93]:
neutral_on_hispanic = get_frequency_neutral_words(
    names_hispanic_emb, neutral_words, dict_vectors, vocab_cleaned, vectors_cleaned, word2idx_cleaned, neighbours_num=100)

neutral_on_hispanic_deb = get_frequency_neutral_words(
    names_hispanic_emb, neutral_words, deb_dict_gender, deb_vocab_gender, deb_vect_gender, deb_word2idx_gender, neighbours_num=100)


 33%|███▎      | 10/30 [00:16<00:32,  1.63s/it]

leader
runner


 47%|████▋     | 14/30 [00:22<00:25,  1.62s/it]

runner
executive


 80%|████████  | 24/30 [00:38<00:09,  1.62s/it]

musician


 93%|█████████▎| 28/30 [00:45<00:03,  1.61s/it]

scientist


100%|██████████| 30/30 [00:48<00:00,  1.62s/it]

janitor


In [ ]:
#frequencies of neighbors
neutra = get_frequency_original_neighbors(
    neutral_words, list_neigh_original_g, dict_g_all, all_g_words, all_g_vectors, w2i_g_all, neighbours_num=50)
df2 = pd.DataFrame(neig_freq2, columns=['word', 'previous_neighbours', 'freq'])
#merge the two dataframes on the word column
#merge df and df average on word
df_merged_punct = df2.merge(df_average, on='word')
#df=pd.concat([df,df_merged], axis=1, keys='words')


In [ ]:
#WEAT
#Distances

2. Words associated with race: 

In [18]:
from operator import itemgetter
#function to get the most biased words in dict_vec_cleaned rated my similarity to the bias direction


def get_most_biased_words_similarity(similarity, n_words=2500):
    #get the absolute values of the similarity values
    similarity = {word: abs(sim) for word, sim in similarity.items()}
    #sort the similarity values
    sorted_similarity = sorted(
        similarity.items(), key=itemgetter(1), reverse=True)
    biased_words = [word for word, bias in sorted_similarity[:n_words*2]]
    neutral_words = [word for word, bias in sorted_similarity[-n_words:]]
    return biased_words, neutral_words


#count how many female_words_emb and male_words_emb are in the biased_words
def count_gendered_words_in_most_biased(female_words, male_words, biased_words):
    count_female=0
    count_male=0
    data={}
    for word in set(female_words):
        if word in set(biased_words):
            count_female+=1

    for word in set(male_words):
        if word in set(biased_words):
            count_male+=1
    
    data.update({'count_female': count_female, 'proportion_female': count_female/len(female_words),
                     'count_male': count_male, 'proportion_male': count_male/len(male_words)})
    return data

In [20]:
biased_words, neutral_words=get_most_biased_words_similarity(similarity_centralized, n_words=40000)

In [ ]:
count_gendered_words_in_most_biased(female_words_emb, male_words_emb, biased_words)


In [ ]:
biased_words_centralized, neutral_words_centralized=get_most_biased_words_similarity(similarity_centralized, n_words=40000)
count_gendered_words_in_most_biased(female_words_emb, male_words_emb, biased_words_centralized)

In [ ]:
_, _, female_simple, male_simple, _ = utils.getting_biased_words(
    simple_gender_bias, def_sets['gender'], 40000, word2idx)

count_gendered_words_in_most_biased(female_words_emb, male_words_emb,female_simple+male_simple )

#### Now let's plot the results

In [ ]:
#turn the similarity scores into a dataframe to find the 20 words with the highest similarity score
df_similarity=pd.DataFrame.from_dict(similarity, orient='index')
df_similarity=df_similarity.rename(columns={0: 'similarity_score'})
df_similarity.sort_values(by=['similarity_score'], ascending=False, inplace=True)
df_similarity_male=df_similarity.head(20)
df_similarity_female=df_similarity.tail(20)

print('Top biased words according to the similarity score:',biased_words[:20])

#turn the centralized similarity scores into a dataframe to find the 20 words with the highest similarity score
df_similarity_centralized=pd.DataFrame.from_dict(similarity_centralized, orient='index')
df_similarity_centralized=df_similarity_centralized.rename(columns={0: 'similarity_score'})
df_similarity_centralized.sort_values(by=['similarity_score'], ascending=False, inplace=True)
df_centralized_female= df_similarity_centralized.head(20)
df_centralized_male=df_similarity_centralized.tail(20)

print('Top biased words according to the centralized similarity score:',biased_words_centralized[:20])

#simple_biased = {word: abs(sim) for word, sim in simple_gender_bias.items()}
df_simple_bias = pd.DataFrame.from_dict(simple_gender_bias, orient='index')
df_simple_bias=df_simple_bias.rename(columns={0: 'simple_bias_score'})
df_simple_bias.sort_values(by=['simple_bias_score'], ascending=False, inplace=True)
df_simple_fem=df_simple_bias.head(20)
df_simple_masc=df_simple_bias.tail(20)
print('Top biased words simple_score', list(df_simple_bias.head(10).index)+list(df_simple_bias.tail(10).index))



In [ ]:
df_centralized_male


In [ ]:
#get a histogram of the similarity.values()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_histogram(similarity, title):
    plt.hist(similarity.values())
    plt.title(title)
    plt.show()

In [ ]:
plot_histogram(similarity, 'Histogram of similarity values')

In [ ]:
plot_histogram(similarity_centralized, 'Histogram of centralized similarity values')


In [ ]:
plot_histogram(simple_gender_bias, 'Histogram of simple gender bias values')


In [ ]:
#get the words in similirity that have values 0
def get_words_with_value_zero(similarity):
    words=[]
    for word in similarity.keys():
        if np.allclose(similarity[word],0.5):
            words.append(word)
    return words

In [ ]:
get_words_with_value_zero(similarity)